# Reactive Pendulum

**NOTE**: This notebook is written in Julia. Julia programming language compiles code on-the-fly and first executation is always very slow due to initial compilation. Sometimes the initial compilation takes several minutes before actual execution of the code (especially GLMakie...). I suggest click `Cell` -> `Run all` and grab a cup of coffee.

In [ ]:
# These two cells take a lot of time for the very first time, especially GLMakie, for interactive plotting
# Installs all necessary packages
import Pkg; Pkg.activate("."); Pkg.instantiate();

In [ ]:
using RxInfer, Rocket, LinearAlgebra, GLMakie, DataStructures

import ReactiveMP: getrecent, messageout, update!
import Rocket: subscribe!

# --- Logger (simple, file-backed) ---
using Dates

const RXINF_LOGFILE = joinpath(@__DIR__, "rxinfer.log")

function rxlog(level::AbstractString, msg)
    ts = Dates.format(Dates.now(), "yyyy-mm-dd HH:MM:SS")
    open(RXINF_LOGFILE, "a") do io
        println(io, "[$ts] $(uppercase(level)) - $msg")
    end
end

rxlog("info", "Logger initialized for ReactiveInferencePendulum")

**Note**: This notebook uses 2 reactive libraries: `Observables.jl` and `Rocket.jl`. `Rocket.jl` has been developed in BIASlab and is highly efficient. `GLMakie.jl`, however, uses `Observables.jl`, because `Rocket.jl` did not exist at the moment. Please, do not confuse `Rocket.jl` observables/actors/subjects and `Observables.jl` observables. The functionality of the `Observables.jl` is very very simple, while `Rocket.jl` is a comprehensive self-contained reactive extensions framework with a lot of extra functionality (and also faster :)).

In [ ]:
# Some utility functionality
# Returns a variable argument function, that shifts a vector and put a new value at the end
function shift(vector, value)
    return (args...) -> begin 
        @inbounds for i in firstindex(vector):lastindex(vector)-1
            vector[i] = vector[i + 1]
        end
        vector[end] = value[]
        return vector
    end
end

# These are utility functions and are not interesting, skip for now
function shift(vector, subject::AbstractSubject)
    return (_) -> begin 
        subscribe!(subject |> take(1), (value) -> shift(vector, value)())
        return vector
    end
end

# Simulation preparation

The very first step in our simulation would be to prepate the pendulum environment that we can play with. Our environment will consist of multiple global parameters, which we will change interactively later on. The parameters are implemented in the `WorldParameters` structure which will be created and shared globally.

In [ ]:
# Our beautiful world parameters
Base.@kwdef mutable struct PendulumWorldParameters
    bob_mass           :: Float64 = 0.2 # grams
    rod_length         :: Float64 = 0.2 # cm
    friction           :: Float64 = 0.2
    gravity            :: Float64 = 9.81
    engine_max_power   :: Float64 = 1.0
    observations_noise :: Float64 = 1e-6
    worlds_clock_Δt    :: Float64 = 1 / 30
end

# Its better not to rerun this cell as it defines the `const` global variable
const parameters = PendulumWorldParameters();

function pendulum_bob_position(angle)
    return Point2f(parameters.rod_length * sin(angle), -parameters.rod_length * cos(angle))
end

## State transition functions

The pendulum differential equations can be represented as a special case of a non-linear state-transition probabilistic model with the following state transition function. In the current simulation we assume that the dynamical model of the world is known. In our simulation we also want to ensure that the engine connected to the pendulum has a limited power. We model such a restriction with the `tanh` function, because its a function with a known inverse mapping.

In [ ]:
# These functions restrict the engine power to a maximum value of `engine_max_power`
function restrict_engine_power(action) 
    return parameters.engine_max_power * tanh(action / parameters.engine_max_power)
end

function state_transition(previous_state, action) 
    # Transition function modeling transition due to gravity, friction and engine control
    (θ, θ̇) = previous_state
    θ̈ = 1 / (parameters.bob_mass * parameters.rod_length ^ 2) * 
        (-parameters.bob_mass * parameters.gravity * parameters.rod_length * sin(θ) - 
            parameters.friction * θ̇ .+ restrict_engine_power(action))
    Δs = (θ̇, θ̈)
    next_state = previous_state .+  Δs .* parameters.worlds_clock_Δt
    return next_state
end

## The implementation of the WORLD

Only one single pendulum exists in our simulated world, which makes our task a bit easier. We implement the world in the `PendulumWorld` structure:

In [ ]:
# BEHOLD THE IMPLEMENTATION OF THE WHOLE WORLD
Base.@kwdef mutable struct PendulumWorld
    pendulum_hidden_state :: Tuple{Float64, Float64} = (0.0, 0.0)
    next_registered_action  = 0.0
    noise_free_observations = RecentSubject(Float64)
    noisy_observations      = RecentSubject(Float64)
    ticks                   = Subject(Bool)
    observations_history    = CircularBuffer(30)
    actions_history         = CircularBuffer(30)
end

# `tick` function is used to move the state of the world further and is independed from any agent
# An agent can only `register` a new action in between with the `register_next_action`
function tick(world::PendulumWorld)
    next_hidden_state = state_transition(world.pendulum_hidden_state, world.next_registered_action)
    stochastic_state  = rand(MvNormalMeanPrecision(collect(next_hidden_state), 1e10 * diageye(2)))
            
    noise_free_observation = first(stochastic_state)
    noisy_observation      = rand(NormalMeanVariance(noise_free_observation, parameters.observations_noise))
        
    # Save history for debugging and plotting
    push!(world.actions_history, restrict_engine_power(world.next_registered_action))
    push!(world.observations_history, noisy_observation)
    
    world.next_registered_action = 0.0
    world.pendulum_hidden_state = (stochastic_state[1], stochastic_state[2])
    
    # Fire tick events 
    next!(world.noise_free_observations, noise_free_observation)
    next!(world.noisy_observations, noisy_observation)
    next!(world.ticks, true)
end

function register_next_action(world::PendulumWorld, action)
    world.next_registered_action = action
    return nothing
end

## The implementation of the AGENT

To implement the pendulum controlling agent we define the probabilistic model of the world with the `@model` macro from **RxInfer**:

In [ ]:
@model function pendulum(T, P, C, m_s_t_min, v_s_t_min, m_u_t_min, v_u_t_min, x_t, m_u, v_u, m_x, v_x, n_alpha, n_theta)
    
    n ~ InverseGamma(n_alpha, n_theta)

    s_t_min ~ MvNormal(mean = m_s_t_min, covariance = v_s_t_min) # Prior for previous state
    u_t_min ~ Normal(mean = m_u_t_min, variance = v_u_t_min)   # Prior for previous action
    u_s_min ~ state_transition(s_t_min, u_t_min)          # Deterministic state transition function
    s_t     ~ MvNormal(mean = u_s_min, precision = P) # Transition uncertainty
    x_t     ~ Normal(mean = dot(C, s_t), variance = n)   # Observational function
    
    s_k_min = s_t
    
    for k in 1:T
        u[k]    ~ Normal(mean = m_u[k], variance = v_u[k])
        u_s[k]  ~ state_transition(s_k_min, u[k])
        s[k]    ~ MvNormal(mean = u_s[k], precision = P)
        x[k]    ~ Normal(mean = dot(C, s[k]), variance = n)
        x[k]    ~ Normal(mean = m_x[k], variance = v_x[k]) 
        s_k_min = s[k]
    end
    return s_t, m_s_t_min, v_s_t_min
end

@meta function pendulum_meta()
    state_transition() -> DeltaMeta(method = Linearization())
end

@constraints function pendulum_constraints()
    q(s_t, x, s, u, n) = q(x, s, u, s_t)q(n)
end

@initialization function pendulum_initialization()
    q(u) = map(agent.mean_control_priors, agent.var_control_priors) do m, v
        NormalMeanVariance(m, v)
    end
    q(n) = InverseGamma(4.0, 1.0)
    q(s_t_min) = MvNormalMeanCovariance([0.0; 0.0], 1e-12*diageye(2))

end

In [ ]:
# Inferred variance is very small, causes numerical instabilities
pick_first_action = (actions) -> begin
    return mean_var(first(actions))
end

pick_current_state = (states) -> begin
    return (agent.mean_current_state_prior, agent.cov_current_state_prior)
end

soft_noise_prior = (noise) -> begin
    μ = mean(noise)
    μ = μ > 0.1 ? 0.1 : μ
    v = 0.1
    α = μ ^ 2 / v + 2
    θ = μ * (α - 1)
    return (α, θ)
end

Next step is to connect the agent with the outside world, for that purpose we create a special `SuperSmartRxInferAgent` structure:

In [ ]:

mutable struct SuperSmartRxInferAgent
    datastream               :: AbstractSubscribable
    rxinfer_engine           :: Union{Nothing, RxInferenceEngine}
    mean_control_priors      :: Vector{Float64}
    var_control_priors       :: Vector{Float64}
    mean_goal_priors         :: Vector{Float64}
    var_goal_priors          :: Vector{Float64}
    mean_current_state_prior :: Vector{Float64}
    cov_current_state_prior  :: Matrix{Float64}
    subscriptions            :: Vector{Teardown}
    execution_time           :: AbstractSubject
    vmp_iterations           :: AbstractSubject
    recent_action            :: AbstractSubject
    free_energy              :: AbstractSubject
    the_goal_in_radians      :: AbstractSubject
    the_goal_variance        :: AbstractSubject

    function SuperSmartRxInferAgent(T::Int, datastream::AbstractSubscribable)
        mean_control_priors = zeros(T)
        var_control_priors  = zeros(T)
        mean_goal_priors    = zeros(T)
        var_goal_priors     = zeros(T)
        mean_current_state_prior = zeros(2)
        cov_current_state_prior  = zeros(2, 2)
        execution_time           = Subject(Float64)
        vmp_iterations           = BehaviorSubject(5)
        recent_action            = RecentSubject(Float64)
        free_energy              = Subject(Float64)
        the_goal_in_radians      = BehaviorSubject(3.14)
        the_goal_variance        = BehaviorSubject(1e-3)
        subscriptions            = []

        agent = new(datastream, nothing,
            mean_control_priors, var_control_priors,
            mean_goal_priors, var_goal_priors,
            mean_current_state_prior, cov_current_state_prior,
            subscriptions, execution_time, vmp_iterations, recent_action,
            free_energy, the_goal_in_radians, the_goal_variance,
        )

        reset!(agent)

        return agent
    end
end

function reset!(agent::SuperSmartRxInferAgent)
    # Use numeric large/small constants, not the functions `huge`/`tiny`
    fill!(agent.mean_control_priors, 0.0)
    fill!(agent.var_control_priors, 1e10)       # previously `huge` (function)
    fill!(agent.mean_goal_priors, 0.0)
    fill!(agent.var_goal_priors, 1e10)          # previously `huge`
    agent.mean_current_state_prior = [ 0.0, 0.0 ]
    agent.cov_current_state_prior = 1e-12 * diageye(2)  # previously `tiny * diageye(2)`

    rxlog("debug", "reset! called: mean_control_priors=$(agent.mean_control_priors), var_control_priors=$(agent.var_control_priors)")

    return nothing
end

function start!(agent::SuperSmartRxInferAgent)
    if !isnothing(agent.rxinfer_engine)
        stop!(agent)
    end

    rxlog("info", "start! called - creating inference engine")

    # Slide helpers: replace `huge` with numeric large value
    shift_mean_control_priors = shift(agent.mean_control_priors, 0.0)
    shift_var_control_priors  = shift(agent.var_control_priors, 1e10)
    shift_mean_goal_priors    = shift(agent.mean_goal_priors, agent.the_goal_in_radians)
    shift_var_goal_priors     = shift(agent.var_goal_priors, agent.the_goal_variance)

    autoupdates = @autoupdates begin
        m_u = shift_mean_control_priors(q(u))
        v_u = shift_var_control_priors(q(u))
        m_x = shift_mean_goal_priors(q(x))
        v_x = shift_var_goal_priors(q(x))
        m_u_t_min, v_u_t_min = pick_first_action(q(u))
        m_s_t_min, v_s_t_min = pick_current_state(q(s_t_min))
        n_alpha, n_theta = soft_noise_prior(q(n))
    end

    initial_forces = map(agent.mean_control_priors, agent.var_control_priors) do m, v
        return NormalMeanVariance(m, v)
    end

    T = length(agent.mean_control_priors)
    iterations_ref = Ref(0)

    vmp_iterations_subscription = subscribe!(agent.vmp_iterations, (vmp_iters) -> begin
        iterations_ref[] = vmp_iters
        rxlog("debug", "vmp_iterations => $vmp_iters")
        if !(isnothing(agent.rxinfer_engine))
            agent.rxinfer_engine.fe_actor.score = zeros(vmp_iters, 30)
            agent.rxinfer_engine.fe_actor.cframe = 1
            agent.rxinfer_engine.fe_actor.cindex = 0
            agent.rxinfer_engine.fe_actor.valid = falses(30)
            rxlog("debug", "Reinitialized fe_actor scoring arrays")
        end
    end)

    engine = infer(
        model = pendulum(
            T=T, 
            P=1e10*diageye(2), 
            C=[1.0, 0.0],
        ),
        meta = pendulum_meta(),
        constraints = pendulum_constraints(),
        datastream = agent.datastream,
        autoupdates = autoupdates,
        initialization = pendulum_initialization(),
        autostart = false,
        returnvars = (:u, ),
        historyvars = (u = KeepLast(), s_t = KeepLast(), n = KeepLast()),
        keephistory = 30,
        free_energy = true,
        free_energy_diagnostics = nothing,
        iterations = iterations_ref,
        events = Val((:before_auto_update, :on_tick))
    )

    on_tick_events = engine.events |> filter(event -> event isa RxInferenceEvent{:on_tick})

    on_tick_subscription = subscribe!(on_tick_events, (args...) -> begin
        slide_msg_idx = 3
        graph     = RxInfer.getmodel(engine.model)
        returnval = RxInfer.getreturnval(graph)[1]
        variable  = RxInfer.getvariable(RxInfer.getvarref(graph, returnval))
        predictive_message = getrecent(messageout(variable, slide_msg_idx))
        (m_s_t_min, v_s_t_min) = mean_cov(predictive_message)

        agent.mean_current_state_prior = m_s_t_min
        agent.cov_current_state_prior = v_s_t_min
        # rxlog("debug", "on_tick updated agent mean/cov priors")
    end)

    recent_action_subscription = subscribe!(engine.posteriors[:u], (actions) -> begin
        next!(agent.recent_action, mode(first(actions)))
    end)

    free_energy_subscription = subscribe!(engine.free_energy, (value) -> begin
        next!(agent.free_energy, value)
    end)

    # rxlog("debug", "All subscriptions created, adding to agent")
    push!(agent.subscriptions, vmp_iterations_subscription)
    push!(agent.subscriptions, on_tick_subscription)
    push!(agent.subscriptions, recent_action_subscription)
    push!(agent.subscriptions, free_energy_subscription)

    agent.rxinfer_engine = engine

    rxlog("debug", "Starting inference engine")
    RxInfer.start(engine)
    rxlog("info", "Agent started")

    return nothing, engine
end

function stop!(agent::SuperSmartRxInferAgent)
    rxlog("info", "stop! called")
    if !isnothing(agent.rxinfer_engine)
        RxInfer.stop(agent.rxinfer_engine)
    end
    foreach(subscription -> unsubscribe!(subscription), agent.subscriptions)
    agent.rxinfer_engine = nothing
    agent.subscriptions = []
    rxlog("info", "Agent stopped and subscriptions removed")
    return nothing
end

# The most exciting part of the notebook

**NOTE**: Note again that Julia initial compilation times are sometimes slow, so the initial execution of this cell takes some time to precompile. Also the `Activate agent` takes some time to precompile the agent, but after the initial compilation the code executes very fast.

Making all run together! Fun fact: 99% code below is just plotting stuff. Slider ranges are controlled in the `SliderGrid` structure. 

P.S. There is some strange bug with "zero range" related to plotting in GLMakie, if that happens simply restart the cell.

In [ ]:
# Setup the space ship's dashboard

# Some naming guidelines
# `r_*` - indicates *R*eactive observable, either from `Observables.jl` or from `Rocket.jl`
# `s_*` - indicates a *S*ubscription
# `b_*` - indicates a *B*utton

# Width of the controls
c_width = 350
c_fontsize = 14

fig = Figure(fontsize=c_fontsize, resolution=(1080, 720))

controls_grid = fig[1:4, 1] = GridLayout()
pendulum_grid = fig[1:4, 2:3] = GridLayout()
auxilary_grid = fig[1:4, 4] = GridLayout()

display(fig, title="Pendulum with RxInfer😎")

free_energy_buffer = CircularBuffer{Float64}(100)

r_world_isrunning = Observable(true) # Is simulation running
r_origin = Observable([Point2f(0, 0), Point2f(0, 0)]) # Position of the origin
r_rod = Observable([Point2f(0, 0), Point2f(0, 0)]) # Position of the rod
r_bob = Observable([Point2f(0, 0)])  # Position of the bob
r_goal = Observable([Point2f(0, 0)]) # Position of the goal
r_observations = Observable(Point2f[]) # Positions of noibsy observations (history)
r_actions = Observable([Point2f(0, 0), Point2f(0, 0)]) # Actions (history)
r_noise_history = Observable(map(_ -> Point2f(0, 0), 1:30)) # Inferred noise (history)
r_noise_bandl = Observable(map(_ -> Point2f(0, 0), 1:30)) # Inferred noise lower band (history)
r_noise_bandu = Observable(map(_ -> Point2f(0, 0), 1:30)) # Inferred noise upper band (history)
r_free_energy = Observable([Point2f(0, 0)])
r_free_energy_acc = Observable([Point2f(0, 0)])

ax_actions_history = Axis(auxilary_grid[1, 1], limits=(0, 30, -1.5, 1.5), title="Agents actions")

lines!(ax_actions_history, r_actions; linewidth=4, color=:blue)

# Dashboard buttons and sliders
b_grid = controls_grid[2, 1] = GridLayout()

b_run = Button(b_grid[1, 1]; label="Activate agent", width=c_width / 2, fontsize=c_fontsize)
b_areset = Button(b_grid[2, 1]; label="Erase agents's memory", width=c_width / 2, fontsize=c_fontsize)
b_stop = Button(b_grid[3, 1]; label="Deactivate agent", width=c_width / 2, fontsize=c_fontsize)

b_corrupt = Button(b_grid[1, 2]; label="Corrupt world's state", width=c_width / 2, fontsize=c_fontsize)
b_wreset = Button(b_grid[2, 2]; label="Reset world's parameters", width=c_width / 2, fontsize=c_fontsize)


sg = SliderGrid(
    controls_grid[1, 1],
    (label="Bob's mass", range=0.15:0.01:0.35, format="{:.3f}g", startvalue=parameters.bob_mass,),
    (label="Rod's Length", range=0.15:0.01:0.25, format="{:.3f}cm", startvalue=parameters.rod_length),
    (label="Maximum engine power", range=0.1:0.1:1.5, format="{:.1f}", startvalue=parameters.engine_max_power),
    (label="Pendulum's friction", range=0.1:0.01:0.3, format="{:.3f}", startvalue=parameters.friction),
    (label="World's gravity", range=1.0:0.1:50.0, format="{:.1f}", startvalue=parameters.gravity),
    (label="Observational noise", range=exp10.(-8.0:0.1:-2), format="{:.6f}", startvalue=parameters.observations_noise),
    (label="VMP iterations", range=1:25, startvalue=5),
    (label="Goal", range=0:0.01:2pi, startvalue=pi),
    (label="Goal variance", range=exp10.(-5.0:0.1:-2), startvalue=exp10(-3)),
    width=c_width,
    tellwidth=true,
    tellheight=true
)

r_mass = sg.sliders[1].value
r_length = sg.sliders[2].value
r_power = sg.sliders[3].value
r_friction = sg.sliders[4].value
r_gravity = sg.sliders[5].value
r_noise = sg.sliders[6].value
r_iters = sg.sliders[7].value
r_goalp = sg.sliders[8].value
r_goalv = sg.sliders[9].value

ax_limits = (-0.3, 0.3, -0.3, 0.3)
ax_pendulum = Axis(pendulum_grid[1, 1], limits=ax_limits, title="Pendulum", aspect=DataAspect())

lines!(ax_pendulum, r_rod; linewidth=5, color=:black)
scatter!(ax_pendulum, r_origin; strokewidth=2, strokecolor=:black, color=:black, markersize=20)
scatter!(ax_pendulum, r_bob; strokewidth=2, strokecolor=:black, color=:black, markersize=map(m -> m * 500, r_mass))
scatter!(ax_pendulum, r_goal; strokewidth=4, strokecolor=:red, color=(:red, 0.2), markersize=120)
scatter!(ax_pendulum, r_observations; strokecolor=:green, color=(:green, :0.2), markersize=map(m -> m * 75, r_mass))

ax_inferred_noise_history = Axis(auxilary_grid[2, 1], yscale=log10, limits=(0, 30, 1e-8, 1.0), title="Estimated noise precision")

lines!(ax_inferred_noise_history, r_noise_history; linewidth=4, color=:blue)
band!(ax_inferred_noise_history, r_noise_bandl, r_noise_bandu, color=(:blue, 0.2))
hlines!(ax_inferred_noise_history, map(e -> [e], r_noise), color=:red)

ax_free_energy_history = Axis(auxilary_grid[3, 1], limits=((0, 100), nothing), xticklabelsvisible=false, yticklabelsvisible=false, title="Bethe Free Energy")
lines!(ax_free_energy_history, r_free_energy)

ax_free_energy_acc_history = Axis(auxilary_grid[4, 1], limits=((1, 15), nothing), xticklabelsvisible=true, yticklabelsvisible=false, title="BFE minimization history")
lines!(ax_free_energy_acc_history, r_free_energy_acc)

## Initialize the environment    

world = PendulumWorld()
agent = SuperSmartRxInferAgent(3, labeled(Val((:x_t,)), combineLatest(world.noisy_observations)))

# Redraw the observations as soon as we have a new data point
s_ticks = subscribe!(world.ticks, (_) -> begin
    r_observations[] = map(angle -> pendulum_bob_position(angle), world.observations_history)
    r_actions[] = map(((index, force),) -> Point2f(index, force), enumerate(world.actions_history))
    r_free_energy[] = map(((index, value),) -> Point2f(index, value), enumerate(free_energy_buffer))

    if !isnothing(agent.rxinfer_engine)
        if length(agent.rxinfer_engine.history[:n]) == 30
            rfem, rfev = mean(free_energy_buffer), clamp(var(free_energy_buffer), 1e-4, Inf)
            if !isnan(rfem) && !isinf(rfem) && !isnan(rfev) && !isinf(rfev)
                ylims!(ax_free_energy_history, clamp(rfem - 20sqrt(rfev), 1e-8, Inf), rfem + 20sqrt(rfev))
            end
            rfeaccmin, rfeaccmax = minimum(agent.rxinfer_engine.free_energy_history), maximum(agent.rxinfer_engine.free_energy_history)
            rfeaccm, rfeaccv = mean(agent.rxinfer_engine.free_energy_history), clamp(var(agent.rxinfer_engine.free_energy_history), 1e-4, Inf)
            if !isnan(rfeaccm) && !isinf(rfeaccm) && !isnan(rfeaccv) && !isinf(rfeaccv)
                xlims!(ax_free_energy_acc_history, 1, length(agent.rxinfer_engine.free_energy_history))
                ylims!(ax_free_energy_acc_history, clamp(rfeaccmin - sqrt(rfeaccv), 1e-8, Inf), rfeaccmax + sqrt(rfeaccv))
            end

            noise_means = map((q_n) -> mean(q_n), agent.rxinfer_engine.history[:n])
            noise_vars = map((q_n) -> var(q_n), agent.rxinfer_engine.history[:n])
            r_free_energy_acc[] = map(((index, value),) -> Point2f(index, value), enumerate(agent.rxinfer_engine.free_energy_history))
            r_noise_history[] = map(((index, mean),) -> Point2f(index, mean), enumerate(noise_means))
            r_noise_bandl[] = map(((index, mean), var) -> Point2f(index, clamp(mean - sqrt(var), 1e-10, Inf)), enumerate(noise_means), noise_vars)
            r_noise_bandu[] = map(((index, mean), var) -> Point2f(index, clamp(mean + sqrt(var), 1e-10, Inf)), enumerate(noise_means), noise_vars)
        end
    end
end)

s_redraw = subscribe!(combineLatest(world.noise_free_observations, agent.the_goal_in_radians), ((angle, goal),) -> begin
    origin_position = Point2f(0.0, 0.0)
    bob_position = pendulum_bob_position(angle)
    r_rod[] = [origin_position, bob_position]
    r_bob[] = [bob_position]
    r_goal[] = [pendulum_bob_position(goal)]
end)

# Register a new action as soon as we have it
s_actions = subscribe!(agent.recent_action, (a) -> register_next_action(world, a))
s_free_energy = subscribe!(agent.free_energy, (v) -> push!(free_energy_buffer, v))

## START THE SHOW!!

# The world runs independently of the agent, but can be force-stopped as well
@async begin
    try
        while isopen(fig.scene) && r_world_isrunning[]
            tick(world)
            sleep(1 / 60)
        end
    catch err
        println("An error happened inside our beautiful world!")
        showerror(stderr, err, catch_backtrace())
    end
    unsubscribe!(s_actions)
    unsubscribe!(s_free_energy)
    unsubscribe!(s_ticks)
    unsubscribe!(s_redraw)
end



# Implement buttons logic
on(b_run.clicks) do clicks
    try
        reset!(agent)
        start!(agent)
    catch err
        rxlog("error", "An error happened inside our beautiful click!: $err")
        println("An error happened inside our beautiful click!")
        showerror(stderr, err, catch_backtrace())
    end
end

on(b_areset.clicks) do clicks
    reset!(agent)
end

on(b_corrupt.clicks) do clicks
    world.pendulum_hidden_state = (0.0, 0.0)
end

on(b_wreset.clicks) do clicks
    local rparams = PendulumWorldParameters()
    r_length[] = parameters.rod_length = rparams.rod_length
    r_mass[] = parameters.bob_mass = rparams.bob_mass
    r_friction[] = parameters.friction = rparams.friction
    r_gravity[] = parameters.gravity = rparams.gravity
    r_power[] = parameters.engine_max_power = rparams.engine_max_power
    r_noise[] = parameters.observations_noise = rparams.observations_noise
    parameters.worlds_clock_Δt = rparams.worlds_clock_Δt
end

on((_) -> stop!(agent), b_stop.clicks)

# Implement sliders logic

on((length) -> begin
    global parameters.rod_length = length
end, r_length)
on((mass) -> begin
    global parameters.bob_mass = mass
end, r_mass)
on((power) -> begin
    global parameters.engine_max_power = power
end, r_power)
on((friction) -> begin
    global parameters.friction = friction
end, r_friction)
on((gravity) -> begin
    global parameters.gravity = gravity
end, r_gravity)
on((noise) -> begin
    global parameters.observations_noise = noise
end, r_noise)
on((iters) -> begin
    next!(agent.vmp_iterations, iters)
end, r_iters)
on((goal) -> begin
    next!(agent.the_goal_in_radians, goal)
end, r_goalp)
on((var) -> begin
    next!(agent.the_goal_variance, var)
end, r_goalv)